Задание 1

In [39]:
import sqlite3
conn = sqlite3.connect('D:/netflix.sqlite')
cursor = conn.cursor()

In [52]:
table1 = cursor.execute("SELECT show_id, actors FROM netflix_titles").fetchall()

In [53]:
task = sqlite3.connect("D:/task1.sqlite")
cursor_task = task.cursor()

Таблица актеры

In [65]:
with task:
    cursor_task.execute("""
        CREATE TABLE actors (
            movie_id INT,
            name TEXT
        );
    """)

In [ ]:
for row in table1:
    movie_id = row[0]
    act = row[1].split(", ")
    data = []
    print(movie_id, act)
    for name in act:
        data.append((int(movie_id), name))
    with task:   
            cursor_task.executemany("INSERT INTO actors VALUES(?, ?)", data)

Таблица фильмы

In [70]:
with task:
    cursor_task.execute("""
        CREATE TABLE movies (
            movie_id INT,
            title TEXT
        );
    """)

In [71]:
table2 = cursor.execute("SELECT show_id, title FROM netflix_titles").fetchall()

In [72]:
with task:
    for row in table2:
        cursor_task.execute("INSERT INTO movies VALUES(?, ?)", (int(row[0]), row[1]))

Таблица актер-фильм

In [ ]:
with task:
    # cursor_task.execute("Union SELECT movie_id, name FROM actors A Exception JOIN movies M on A.movie_id = M.movie_id")
    sq = cursor_task.execute("SELECT DISTINCT A.movie_id, A.name, m.title FROM actors AS a INNER JOIN movies AS m on a.movie_id = m.movie_id").fetchall()
    for row in sq:
        # cursor_task.execute("INSERT INTO actors_movies VALUES(?, ?, ?)", row)
        data = (row[0], row[1], row[2])
        cursor_task.execute("INSERT INTO actors_movies VALUES(?, ?, ?)", data)
        

In [97]:
with task:
    pair = cursor_task.execute("SELECT a1.actor_name AS actor1, a2.actor_name AS actor2, COUNT(*) AS count FROM \
                        actors_movies a1 JOIN actors_movies a2 ON a1.title = a2.title AND a1.actor_name < a2.actor_name\
                         GROUP BY actor1, actor2 ORDER BY count DESC LIMIT 1;").fetchall()
    print(pair)

[('John Paul Tremblay', 'Robb Wells', 15)]
